In [1]:
import requests
import os
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import earthpy.plot as ep
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from tqdm import tqdm
from rasterio.plot import plotting_extent
import csv
import io
from contextlib import redirect_stdout
from shapely.geometry import Point, box


data_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'Chile-Glaciers')


In [2]:
import ee
import geemap

ee.Initialize()

In [3]:
glacier_shape_path = os.path.join(data_dir, 'data', 'glims_buffer', 'glims_buffer.shp')
study_area_path = os.path.join(data_dir, 'data', 'studyareas', 'studyareas.shp')
small_area_file = os.path.join(data_dir, 'data', 'studyareas', 'small_study.geojson')

glaciers = gpd.read_file(glacier_shape_path)
study_areas = gpd.read_file(study_area_path).to_crs(glaciers.crs)
small_box = box(350000, 6200000, 500000, 6400000)
small_box_gdf = gpd.GeoDataFrame(geometry=[small_box], crs=glaciers.crs)
small_glaciers = glaciers.clip(small_box_gdf)
small_box_gdf.to_crs('EPSG:4326').to_file(small_area_file)

In [4]:
Map = geemap.Map()
# We will use the Sentinel-2 Surface Reflection product.
# This dataset has already been atmospherically corrected

country = geemap.geojson_to_ee(small_area_file)
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
# Function to mask clouds S2
def maskS2srClouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)


  # Filter Sentinel-2 collection
s2Filt = s2.filterBounds(country).filterDate(
                         '2023-01-01','2023-02-28').filterMetadata(
                         'CLOUDY_PIXEL_PERCENTAGE', 'less_than',50).map(maskS2srClouds)

  # print('Sentinel-2 Filtered collection',s2Filt)

  # Composite images
s2composite = s2Filt.median().clip(country); # can be changed to mean, min, etc
s2composite


In [5]:
# # Add composite to map
Map.addLayer(s2composite,{'bands':['B4', 'B3', 'B2'], 'min':0.02, 'max':0.3,
                          'gamma':1.5},'Sentinel-2 2023 composite')
Map.setCenter(-69, -33, 8)
# # Test another band combination (SWIR 1, NIR, Blue)
Map.addLayer(s2composite,{'bands':['B11', 'B8', 'B2'], 'min':0, 'max':0.6}, 'Sentinel-2 2023 composite SWIR/NIR/Blue')
Map

Map(center=[-33, -69], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…